<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Próbkowanie</font>

<font size="4">Krzysztof Dyba</font>

# Próbkowanie

Próbkowanie (*sampling*) to proces wybierania (selekcji) podzbioru lokalizacji danych przestrzennych z większego obszaru geograficznego w celu analizy cech całego obszaru. Stworzona reprezentatywna próba danych, może zostać wykorzystana do wyciągnięcia wniosków na temat populacji, przy jednoczesnym zminimalizowaniu czasu, kosztów i wysiłku wymaganego do zebrania i przetworzenia tych danych.

**Rodzaje próbkowania:**

- losowe -- lokalizacje wybierane są całkowicie przypadkowo
- stratyfikowane (łac. *stratum* -- warstwa) -- obszar dzielony jest na podgrupy (*stratum*) na podstawie pewnych cech (np. użytkowanie terenu, wysokość). Następnie pobierane są losowe próbki w obrębie każdego *stratum*, co zapewnia zbalansowaną reprezentację różnych podgrup
- reguralne / systematyczne -- próbki wybierane są w określonych i równomiernych odsępach, np. co 100 m.
- celowe -- próbki wybierane są na podstawie wiedzy badacza i określonych kryteriów

## Próbkowanie losowe

Istnieje wiele sposobów losowego doboru próbek używając różnych rozkładów prawdopodobieństwa (np. jednostajnego, normalnego, dwumianowego czy Poissona) w zależności od charakterystyki danych i konkretnych wymagań analizy. W przypadku danych przestrzennych wykorzystywane są specjalistyczne rozkłady przestrzenne, które są bardziej złożone od tradycyjnych ze względu na uwzględnienie relacji przestrzennych (patrz pakiet [spatstat.random](https://cran.r-project.org/web/packages/spatstat.random/index.html) w R).

W najprostszym przypadku należy wykorzystać losowanie ze zwracaniem, gdzie jako obiekt wejściowy definiujemy listę, która reprezentuje minimalny oraz maksymalny zasięg przestrzenny danej osi, np. `x_coords = list(range(xMinimum, xMaximum + 1))`. Z racji, iż dane rastrowe są reprezentacją dyskretną obszarów badań, trzeba zdefiniować interwał próbkowania, np. co 10 m, aby uniknąć sytuacji, w której próbki pochodzą z identycznej lokalizacji (komórki). Zakres przestrzenny możemy zdefiniować na dwa sposoby -- dla całego rastra lub dla określonego obszaru. Jeśli korzystamy z QGIS, to dodatkowo możemy, pobrać aktualnie wyświetlany zasięg warstwy.

Uwaga! Generowanie liczb losowych jest procesem losowym, to znaczy że za każdym razem, gdy kod zostanie wykonany, to otrzymamy inną sekwencję liczb. Aby uzyskać tę samą sekwencję liczb, należy ustawić ziarno losowości (`random.seed()`) najlepiej na początku skryptu, dzięki czemu wyniki będą powtarzalne.

In [1]:
import random
random.seed(999) # ziarno losowości

x_coords = list(range(0, 100 + 1, 10))
# losowanie ze zwracaniem
x_coords = random.choices(x_coords, k = 5)
print(x_coords)

[80, 0, 90, 60, 50]


Do storzenia punktu wymagana jest para współrzędnych X (długość geograficzna) i Y (szerokość geograficzna) oraz klasa `QgsPointXY`. Wygnerowane punkty możemy przechowywać w liście (co jest prostszym rozwiązaniem) lub stworzyć nową warstwę wektorową (patrz temat "Przetwarzanie danych wektorowych: Tworzenie nowej warstwy").

In [2]:
from qgis.core import QgsPointXY

x_coords = list(range(0, 100 + 1, 10))
x_coords = random.choices(x_coords, k = 5)
y_coords = list(range(0, 100 + 1, 10))
y_coords = random.choices(y_coords, k = 5)

points = []
for x, y in zip(x_coords, y_coords):
    points.append(QgsPointXY(x, y))

print(points)

[<QgsPointXY: POINT(10 10)>, <QgsPointXY: POINT(80 100)>, <QgsPointXY: POINT(30 20)>, <QgsPointXY: POINT(80 20)>, <QgsPointXY: POINT(90 60)>]


## Próbkowanie stratyfikowane

In [ ]:
# TODO
# 1. wygenerowanie punkty w poligonie
# 2. sprawdzenie czy punkt znajduje się w poligonie

## Próbkowanie regularne

In [22]:
from qgis.core import QgsLineString, QgsGeometry

pt1 = QgsPointXY(10, 20)
pt2 = QgsPointXY(50, 70)
line = QgsGeometry(QgsLineString([pt1, pt2])) # QgsGeometry.fromPolylineXY()
print(line)

line_length = line.length()
print(line_length)

<QgsGeometry: LineString (10 20, 50 70)>
64.03124237432849


In [26]:
interval = 10
points = []
distance = 0
while distance <= line_length:
    point = line.interpolate(distance)
    points.append(point.asPoint())
    distance = distance + interval
print("Liczba punktów:", len(points))

Liczba punktów: 7


In [ ]:
# TODO
# sprawdzenie kierunku linii

# Zadania:

11) **Analiza geomorfometryczna.** Dokonaj analizy ukształowania terenu (plik `DEM.tif`) używając dwóch transektów północ-południe oraz wschód-zachód. Wynik zaprezentuj na rycinie (na osi X powinna znajdować się odległość wyrażona w metrach). Zastanów się również, co zrobić w przypadku, gdy komórki nie posiadają wartości (*nan*).

12) **Analiza teledetekcyjna.** Wygeneruj losową próbę i następnie dla dwóch wybranych kanałów spektralnych z Landsat 8/9:
- wykonaj wykres rozrzutu uwzględniając przezroczystość punktów oraz linię `x = y`,
- oblicz statysytki opisowe z wartości różnic między tymi kanałami.

  Dodatkowo można obliczyć współczynnik korelacji Pearsona (`numpy.corrcoef()`).
  
13) **Analiza geoinformacyjna.** Zaimplementuj funkcję losowania stratyfikowanego, która zagwarantuje, że została zwrócona dokładna liczba punktów wskazana przez użytkownika i wynik zwróci jako warstwę wektorową. Wykorzystaj dane wektorowe `powiaty.gpkg`.